In [1]:
import numpy as np
from scipy import stats 

## Тестовое задание:  
Было запущено 2 рекламные компании одного и того же продукта.  
Первые результаты были следующими:  
* среди мужской аудитории было 3000 показов рекламы и 130 кликов  
* среди женской аудитории было 980 показов и 50 кликов  

Можно ли с вероятностью 95% утверждать, что в дальнейшем конверсия среди женской аудитории будет выше, чем среди мужской.

## Ответ  

Данная задача является стандартным представителем однофакторного дисперсионного анализа, на одну переменную $Y$ влияет один фактор, наблюдаемый в $k$ выборках. В данных задачах проверяется гипотеза  $H_0$ о равенстве __средних значений__ по каждой выборке. Разница лишь в том, что здесь среднее значение указано не явно, как и шкала измерения исследуемых выборок, а также сформулированна правосторонняя обратная гипотеза, вместо нулевой.   
Данная выборки указывают на вероятность клика (_"дальнейшая конверсия"_) среди мужской и женской аудиторий.

Итак, давайте сформулируем задачу явно  
__Дано__:
* $H_0$ (_нулевая гипотеза_) - среднее значение дальнейшей конверсии среди мужской и женской аудиторий равны или незначительно отличаются;
* $\alpha$ (_уровень значимости_) - `0.05` (или `5%`)  

## Как будем решать

В предположении верности гипотезы  $H_0$ статистика $T$ имеет распределение Фишера. Для выбранного уровня значимости $\alpha$ можно рассчитать критическое значение  $F_{crit}$, равное квантилю порядка $1-\alpha$. Если $T>F_{crit}$, то гипотеза $H_0$ отвергается.

### Немного теории и формул

Формула определения T-статистики:
$$T=\frac{\sigma_b^2}{\sigma_w^2}$$

где $\sigma_b^2 \, и \, \sigma_w^2$ - несмещённые оценки дисперсии:
$$\sigma_b^2=\frac{S_b^2}{k-1},\enspace \sigma_w^2=\frac{S_w^2}{n-k}$$

где:
* $k$ - количество выборок (упоминали ранее, но считаю, что можно повторить);
* $n$ - общее количество наблюдений во всех выборках;
* $S_b^2$ - сумму квадратов отклонений средних значений выборок от общего среднего значения;
* $S_w^2$ - остаточная сумма квадратов отклонений.

$$S_b^2=\sum_{i=1}^k(\overline{y_i}-\overline{y})^2\cdot n_i,\enspace 
S_w^2=\sum_{i=1}^k\sum_{j=1}^{n_i}(y_{ij}-\overline{y_i})^2$$

### Используя все эти знания, можно решить поставленную задачу

Для удобства приму переменную $y_i$ за дальнейшую конверсию выборок, где $i$ - обозначение одной из выборок (пусть $m$ - мужская аудитория (male), $f$ - женская аудитория (female)).  
* $\overline{y_m}$ (Среднее значение дальнейшей конверсии среди мужской аудитории) $=\frac{130}{3000}$;  
* $\overline{y_f}$ (Среднее значение дальнейшей конверсии среди женской аудитории) $=\frac{50}{980}$;
* $n$ - $3980$ (3000 мужской аудитории и 980 женской аудитории);
* $k$ - $2$ (мужская и женская)

In [2]:
n_m = 3000
n_f = 980
n_all = n_m + n_f
k = 2
alpha = 0.05
conv_m_mean = 130 / n_m # conversion male mean
conv_f_mean = 50 / n_f # conversion female mean

males = np.hstack((np.zeros(int(n_m * (1 - conv_m_mean))), np.ones(int(n_m * conv_m_mean))))
females = np.hstack((np.zeros(int(n_f * (1 - conv_f_mean))), np.ones(int(n_f * conv_f_mean))))
_all = np.concatenate((males, females))

conv_m_mean, conv_f_mean

(0.043333333333333335, 0.05102040816326531)

In [3]:
# Вычислим среднее значение обеих выборок. Оно пригодится нам для дальнейших расчётов

conv_mean = _all.mean()
conv_mean # conversion all mean

0.04522613065326633

In [4]:
S_b = (conv_m_mean - conv_mean) ** 2 * n_m + (conv_f_mean - conv_mean) ** 2 * n_f
S_b

0.043650223908658935

In [5]:
S_w = ((males - conv_m_mean) ** 2).sum() + ((females - conv_f_mean) ** 2).sum()
S_w

171.81564625850336

Можно проверить верность вычисления сумм с помощью суммы квадратов отклонений наблюдений от общего среднего:

$$S^2=\sum_{i=1}^k\sum_{j=1}^{n_i}(y_{ij}-\overline{y})^2$$

In [6]:
S = ((np.concatenate((males, females)) - conv_mean) ** 2).sum()
print(S)
print(S_w + S_b) # незначительное отколение также можно списать на ограничение компьютерных вычислений.

171.85929648241208
171.85929648241202


In [7]:
sigma_b = S_b / (k - 1)
sigma_w = S_w / (n_all - k)
sigma_b, sigma_w

(0.043650223908658935, 0.04319146462003604)

In [8]:
T = sigma_b / sigma_w
T

1.0106215265599046

In [9]:
F_crit = stats.f.ppf(1 - alpha, k-1, n_all-k)
F_crit

3.8437975159688866

In [10]:
p = 1 - stats.f.cdf(T, k-1, n_all-k)
p

0.31481513123106797

Видно, что $T < F_{crit}$, что свидетельствует о том, что отличие средних значений дальнейшей конверсии разных полов не является статистически значимым.

## Ответ на поставленный вопрос
Нельзя с вероятностью 95% утверждать, что в дальнейшем конверсия среди женской аудитории будет выше, чем среди мужской.

## Вариант решения побыстрее

In [11]:
stats.f_oneway(males, females)

F_onewayResult(statistic=1.0106215265599028, pvalue=0.3148151312289337)